<a href="https://colab.research.google.com/github/AUT-Student/BigData-HW1/blob/main/BigData_HW1_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

# Download File

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id='1-OCBGBtKoY_PadKHcXDyWxHQ2BS8Nulo'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('hw1.zip')

!unzip hw1.zip

Archive:  hw1.zip
   creating: hw1-files/
   creating: hw1-files/q3/
  inflating: hw1-files/q3/patches.csv  
  inflating: hw1-files/q3/lsh.py     
   creating: hw1-files/q1/
  inflating: hw1-files/q1/dataset1.txt  
   creating: hw1-files/q1/.ipynb_checkpoints/
   creating: hw1-files/q2/
  inflating: hw1-files/q2/games_library.txt  
   creating: hw1-files/.ipynb_checkpoints/


# PySpark Setting

In [3]:
!pip install pyspark
!pip install -q findspark

In [4]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.7/dist-packages/pyspark'

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [6]:
spark

# Load Dataset

In [15]:
#df = spark.read.format("csv").option("sep","\t").option("header", False).load("/content/hw1-files/q1/dataset1.txt")

In [53]:
def convert_string_to_list(string):
  if string=="":
    return []
  else:
    return [int(item) for item in string.split(",")]

In [54]:
import pandas as pd
df = pd.read_csv("/content/hw1-files/q1/dataset1.txt", sep="\t",names=["id", "friend_id"])

In [55]:
df = df.fillna("")
df["friend_id"] = df["friend_id"].apply(convert_string_to_list)

In [78]:
rdd=spark.sparkContext.parallelize(df.to_dict('records'), 2)